In [ ]:
import yfinance as yf
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, RepeatVector, Concatenate, Attention
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# Apple hisse senedi verilerini çekme
data = yf.download('AAPL', start='2018-01-01', end='2023-01-01')

# Kullanılacak özellikleri seçme
features = ['Open', 'High', 'Low', 'Close', 'Volume']
data = data[features]

# Veriyi normalize etme
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Veriyi dizilere bölme
def create_sequences(data, seq_length, pred_length):
    X, y = [], []
    for i in range(len(data)-seq_length-pred_length):
        X.append(data[i:(i+seq_length)])
        y.append(data[(i+seq_length):(i+seq_length+pred_length)])
    return np.array(X), np.array(y)

seq_length = 60  # Son 60 günü kullanarak
pred_length = 15  # Önümüzdeki 15 günü tahmin etme
X, y = create_sequences(data_scaled, seq_length, pred_length)


In [ ]:
# Encoder
encoder_inputs = Input(shape=(seq_length, len(features)))
encoder_lstm = LSTM(100, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = RepeatVector(pred_length)(encoder_outputs)
decoder_lstm = LSTM(100, return_sequences=True)
decoder_outputs = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Dikkat Mekanizması
attention_layer = Attention(use_scale=True)
attention_outputs = attention_layer([decoder_outputs, encoder_outputs])
decoder_concat_input = Concatenate(axis=-1)([decoder_outputs, attention_outputs])

# TimeDistributed Dense
decoder_dense = TimeDistributed(Dense(len(features)))
decoder_outputs = decoder_dense(decoder_concat_input)

# Modeli tanımla
model = Model(inputs=encoder_inputs, outputs=decoder_outputs)
model.compile(optimizer='adam', loss='mse')

print(model.summary())


In [ ]:
#modelin pdf halini çıkarma

!pip install pydot graphviz
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True , show_layer_names=True)


In [ ]:
from PIL import Image
image = Image.open('model.png')

In [ ]:
model.fit(X, y, batch_size=32, epochs=200, validation_split=0.2)


In [ ]:
# Son veri noktasını kullanarak geleceği tahmin etme
last_sequence = data_scaled[-seq_length:]
last_sequence = np.expand_dims(last_sequence, axis=0)
predicted = model.predict(last_sequence)

# Tahminleri gerçek değerlere dönüştürme
predicted = scaler.inverse_transform(predicted[0])


In [ ]:
#tahtmin edilen verileri görselleştirme
import matplotlib.pyplot as plt

plt.plot(data.index[-pred_length:], predicted[:, 3], label='Tahmin')
plt.legend()
plt.show()


In [ ]:
plt.plot(data.index[-pred_length:], data['Close'][-pred_length:], label='Gerçek')
plt.legend()
plt.show()

In [ ]:
#predictionsun mum grafiklerini çizdirme

import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=data.index[-pred_length:],
                open=predicted[:, 0],
                high=predicted[:, 1],
                low=predicted[:, 2],
                close=predicted[:, 3])])

fig.show()


In [ ]:
#gerçek verilerin mum grafiklerini çizdirme
fig = go.Figure(data=[go.Candlestick(x=data.index[-pred_length:],
                open=data['Open'][-pred_length:],
                high=data['High'][-pred_length:],
                low=data['Low'][-pred_length:],
                close=data['Close'][-pred_length:])])

fig.show()